In [2]:
import numpy as np
import matplotlib
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = np.loadtxt('GMDataSet.txt')
print(data)
print(data.shape)

[[ 0.14312972  5.99923589  3.28117724  4.85942456  2.76031438]
 [-0.22836165  6.33846814  1.25655149  3.04360083  3.34139874]
 [ 0.52717013  5.51795428  1.72203474  4.75461936  3.74163305]
 ..., 
 [ 0.59767314  4.50230892  2.83994059  6.70687526  2.13694945]
 [ 1.9429948   4.78106102  2.68362201  5.53314876  2.81631195]
 [ 1.1256556   4.39106783  2.06027088  3.01530772  1.0690636 ]]
(1000, 5)


In [4]:
def initialize(data,k):
    expectation = np.zeros((k,data.shape[0]))
    ppi = np.zeros(shape=(k,1))
    miu = np.zeros(shape=(k,data.shape[1]))
    sigma = np.zeros(shape=(k,data.shape[1],data.shape[1]))
    for i in range(0,k):
        ppi[i][0] = float(1/k)
        n = int(data.shape[0]/k)
        for j in range(0,data.shape[1]):
            miu[i][j] = (sum(data[i*n:(i+1)*n,j]))/n
        sum_cov = np.zeros(shape=(data.shape[1],data.shape[1]))
        for x in range(i*n,(i+1)*n):
            datax = data[x] + np.zeros([1,5])
            sum_cov += (datax - miu[i]).T @ (datax - miu[i])
        sigma[i] = sum_cov/n
    return ppi,miu,sigma,expectation       

In [24]:
def e_step(data,ppi,miu,sigma,expectation,k):
    for i in range(0,data.shape[0]):
        denominator = 0
        numerator = 0
        datax = data[i] + np.zeros([1,5])
        for j in range(0,k):
            sigma_inv = np.linalg.inv(sigma[j]+0.0001*np.eye(sigma[j].shape[0]))
            denominator += math.exp(-1/2*((datax-miu[j])@sigma_inv@(datax-miu[j]).T))
        for j in range(0,k):
            sigma_inv = np.linalg.inv(sigma[j]+0.0001*np.eye(sigma[j].shape[0]))
            numerator = math.exp(-1/2*((datax-miu[j])@sigma_inv@(datax-miu[j]).T))
            expectation[j,i] = numerator / denominator
            #print(expectation[j,i])
    return expectation

In [6]:
def m_step(data,ppi,miu,sigma,expectation,k):
    for j in range(0,k):
        numerator_miu  = 0
        numerator_pi = 0
        numerator_sigma = 0
        denominator = 0
        for i in range(0,data.shape[0]): # update miu pi sigma
            datax = data[i] + np.zeros([1,5])
            numerator_miu += expectation[j,i] * datax
            numerator_pi += expectation[j,i]
            numerator_sigma += expectation[j,i] * ((datax-miu[j]).T@(datax-miu[j]))
            denominator += expectation[j,i]
        miu[j] = numerator_miu / denominator
        ppi[j] = numerator_pi / data.shape[0]
        sigma[j] = numerator_sigma / denominator
    return ppi,miu,sigma            

In [7]:
def em(data,k,iter_num,epsilon):
    ppi,miu,sigma,expectation = initialize(data,k)    
    for i in range(0,iter_num):
        old_ppi = np.zeros(shape=(k,1))
        old_miu = np.zeros(shape=(k,data.shape[1]))
        old_sigma = np.zeros(shape=(k,data.shape[1],data.shape[1]))
        old_ppi = old_ppi + ppi
        old_miu = old_miu + miu
        old_sigma = old_sigma + sigma
        expectation = e_step(data,ppi,miu,sigma,expectation,k)
        ppi,miu,sigma = m_step(data,ppi,miu,sigma,expectation,k)
        if abs(np.sum(abs(old_ppi - ppi)))<epsilon and abs(np.sum(abs(old_miu - miu)))<epsilon and abs(np.sum(abs(old_sigma - sigma)))<epsilon:
            print('convergence')
            print ('iteration times:',i)           
            return ppi,miu,sigma
    print('reach iteration times')   
    return ppi,miu,sigma

In [34]:
new_ppi,new_miu,new_sigma = em(data,3,1000,0.0001)
print(new_ppi)

convergence
iteration times: 59
[[ 0.00121564]
 [ 0.57445725]
 [ 0.42432711]]


In [35]:
print(new_miu)

[[  5.83828151e-01   4.19328414e+00   3.38978970e+00   5.10272710e+00
    3.31517071e+00]
 [  4.74024961e-03   4.96942397e+00   2.55305411e+00   4.11288623e+00
    3.06560502e+00]
 [  1.16415031e+00   5.03638394e+00   2.47070467e+00   3.75665143e+00
    1.84639988e+00]]


In [36]:
print(new_sigma)

[[[  1.61636928e-02  -9.60487074e-04  -7.50576366e-03   1.36074736e-02
    -1.94716641e-02]
  [ -9.60487074e-04   5.70745455e-05   4.46011259e-04  -8.08590137e-04
     1.15705501e-03]
  [ -7.50576366e-03   4.46011259e-04   3.48537236e-03  -6.31875912e-03
     9.04185142e-03]
  [  1.36074736e-02  -8.08590137e-04  -6.31875912e-03   1.14555097e-02
    -1.63923034e-02]
  [ -1.94716641e-02   1.15705501e-03   9.04185142e-03  -1.63923034e-02
     2.34566263e-02]]

 [[  1.26372123e+00   1.58729468e-02   4.21472865e-01   9.63178107e-01
     3.84657677e-03]
  [  1.58729468e-02   5.52574258e-01  -2.03237364e-02  -5.44766191e-02
     8.79090159e-03]
  [  4.21472865e-01  -2.03237364e-02   6.17940595e-01   4.73058303e-01
    -1.68321685e-02]
  [  9.63178107e-01  -5.44766191e-02   4.73058303e-01   1.68661508e+00
     1.24816234e-03]
  [  3.84657677e-03   8.79090159e-03  -1.68321685e-02   1.24816234e-03
     4.60174206e-01]]

 [[  4.03153622e-01   8.34294632e-03   4.69348091e-02   7.47410220e-02
     